# 1. Read and pre-process the data

## 1.1 Read data

This is the first part of the code and is designed for checking and cleaning the data. The data used is taken from Alberta Water Quality Data Portal with focus on water matrix (0) and long-term and tributary monitoring stations:
Source for getting data: https://environment.extranet.gov.ab.ca/apps/WaterQuality/dataportal/DataDownload/Index/

In [2]:
import pandas as pd

In [75]:
# Specify parse date:
parse_dates = ['SampleDateTime']

# Now read the data
data = pd.read_csv('../data/Water Quality-2025-03-08 172848.csv', dtype=
{
 'ProjectNumber': 'string',
 'SampleNumber': 'string',
 'ContinentalRiverBasinCode': 'category',
 'RiverBasinCode': 'category',
 'RiverSubBasinCode': 'category',
 'StationTypeCode': 'string',
 'StationNumber': 'category',
 'Station': 'category',
 'LatitudeDecimalDegrees': 'float64',
 'LongitudeDecimalDegrees': 'float64',
 'SampleMatrixCode': 'category',
 'SampleTypeCode': 'category',
 'CollectionCode': 'category',
 'QCSampleFlag': 'category',
 'SampleComment': 'string',
 'SampleDateTime': 'string',
 'VmvCode': 'string',
 'VariableCode': 'string',
 'VariableName': 'category',
 'MeasurementFlag': 'category',
 'MeasurementValue': 'float64',
 'UnitCode': 'category',
 'SampleDetectLimit': 'string',
 'MeasurementComment': 'string',
 'MeasurementQualifier': 'string',
 'MeasurementQualifierDescription': 'string',
 'MeasurementQualifierComment': 'string',
 'MethodCode': 'string',
 'MethodDetectionLimit': 'float64',
 'LabCode': 'category'
 }
 , na_values=['', 'NaN', 'NULL', 'N/A', 'NA', 'null'])

In [76]:
# convert the SampleDateTime column to datetime
data['SampleDateTime'] = pd.to_datetime(data['SampleDateTime'], format='%m/%d/%Y %H:%M:%S', errors='coerce')

# Convert SampleDetectLimit to numeric
data['SampleDetectLimit'] = pd.to_numeric(data['SampleDetectLimit'], errors='coerce')

# 1.2 Filter the data

In [77]:
# Check missingness for each column and arrange in descending order
data.isnull().sum().sort_values()

ProjectNumber                            0
VmvCode                                  0
SampleDateTime                           0
MeasurementValue                         0
QCSampleFlag                             0
MethodCode                               0
SampleTypeCode                           0
SampleMatrixCode                         0
VariableCode                             0
LongitudeDecimalDegrees                  0
Station                                  0
StationNumber                            0
StationTypeCode                          0
RiverSubBasinCode                        0
RiverBasinCode                           0
ContinentalRiverBasinCode                0
SampleNumber                             0
LatitudeDecimalDegrees                   0
VariableName                             0
LabCode                                288
UnitCode                             73616
CollectionCode                      200683
MethodDetectionLimit                386180
SampleComme

**Note:** it is the user's choice to not include the data without unit codes. I chose to eliminate them as they really create ambiguity in how usable the data will be. Just as a more general rule, we apply SampleDateTime, MeasurementValue, StationNumber, and VariableName. 

In [97]:
data = data.dropna(subset = ['UnitCode', 'MeasurementValue', 'SampleDateTime', 'StationNumber', 'VariableName'])
data = data.dropna(how = 'all') # drop rows where all elements are NaN

In [ ]:
duplicates = data.duplicated(subset=['SampleDateTime', 'StationNumber', 'VariableName', 'MeasurementValue', 
                                     'UnitCode', 'VmvCode', 'SampleNumber', 'LabCode'], 
                             keep=False)
                             
# write duplicate rows to csv   
# data[duplicates].to_csv('../output/duplicate_rows.csv', index=False)

Basically there is no duplicate row based on the analysis of duplicates. 

## 1.3 EDA

More to do: harmonize the units for each parameter.

In [78]:
data.describe()

,LatitudeDecimalDegrees,LongitudeDecimalDegrees,SampleDateTime,MeasurementValue,SampleDetectLimit,MethodDetectionLimit
count,2.283487e+06,2.283487e+06,2283487,2.283487e+06,1676152.0,1.897307e+06
mean,5.250656e+01,-1.136527e+02,2012-11-14 19:27:48.970451456,5.697865e+01,0.428427,4.913207e-01
min,4.902673e+01,-1.188047e+02,1959-02-20 14:00:00,-2.130000e+02,0.0,0.000000e+00
25%,5.035310e+01,-1.144871e+02,2007-06-18 08:55:00,2.000000e-02,0.005,6.000000e-03
50%,5.208902e+01,-1.134421e+02,2016-03-15 09:30:00,5.000000e-01,0.05,5.000000e-02
75%,5.401297e+01,-1.124759e+02,2020-08-04 12:30:00,8.500000e+00,0.2,3.000000e-01
max,5.844722e+01,-1.100297e+02,2024-12-12 14:45:00,1.330000e+06,400.0,4.000000e+02
std,2.508057e+00,1.993700e+00,NaN,1.787225e+03,1.851364,1.648276e+00
